# 제8장 한국어 WaveNet 음성 합성 시스템의 구현

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r9y9/ttslearn/blob/master/notebooks/ch08_Recipe-WaveNet.ipynb)

Google colab에서 실행하는 예상 소요 시간: 5시간

이 노트북의 레시피 설정은 Google Colab에서 실행할 때 시간 초과를 피하기 위해 학습 조건을 책에 나열된 설정에서 일부 수정했음을 기억하십시오 (배치 크기 줄이기 등).
참고로 책에 기재된 조건으로 저자(야마모토)가 레시피를 실행한 결과를 아래에서 공개하고 있습니다.

- Tensorboard logs: https://tensorboard.dev/experiment/yXyg9qgfQRSGxvil5FA4xw/
- exp 디렉토리 (학습 모델, 합성 음성 포함) : https://drive.google.com/file/d/1Z09yCCAKyKOUU3Zsdxs0mZ1Q-TCPqFHA/view?usp=sharing (135.2 MB)

## 준비

### Google Colab을 사용하는 경우

Google Colab에서 이 노트북을 실행하려면 메뉴의 '런타임 -> 런타임 시간 변경'에서 '하드웨어 가속기'를 **GPU**로 변경하세요.

### Python version

In [ ]:
!python -VV

### ttslearn 설치

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

## 8.1 이 장의 일본어 음성 합성 시스템 구현

### 학습된 모델을 이용한 음성 합성

In [ ]:
from ttslearn.wavenet import WaveNetTTS
from tqdm.notebook import tqdm
from IPython.display import Audio

engine = WaveNetTTS()
wav, sr = engine.tts("ウェーブネットにチャレンジしましょう！", tqdm=tqdm)
Audio(wav, rate=sr)

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8,2))
librosa.display.waveshow(wav.astype(np.float32), sr, ax=ax)
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

### レシピ実行の前準備

In [ ]:
%%capture
from ttslearn.env import is_colab
from os.path import exists

# pip install ttslearn은 레시피를 설치하지 않으므로 수동으로 다운로드
if is_colab() and not exists("recipes.zip"):
    !curl -LO https://github.com/r9y9/ttslearn/releases/download/v{ttslearn.__version__}/recipes.zip
    !unzip -o recipes.zip

In [ ]:
import os
# recipe 디렉토리로 이동
cwd = os.getcwd()
if cwd.endswith("notebooks"):
    os.chdir("../recipes/wavenet/")
elif is_colab():
    os.chdir("recipes/wavenet/")

In [ ]:
import time
start_time = time.time()

### 패키지 임포트

In [ ]:
%pylab inline
%load_ext autoreload
%load_ext tensorboard
%autoreload
import IPython
from IPython.display import Audio
import tensorboard as tb
import os

In [ ]:
# 수치 연산
import numpy as np
import torch
from torch import nn
# 음성 파형 불러오기
from scipy.io import wavfile
# 풀 컨텍스트 라벨, 질문 파일 로드
from nnmnkwii.io import hts
# 음성 분석
import pyworld
# 음성 분석, 시각화
import librosa
import librosa.display
import pandas as pd
# 파이썬에서 배우는 음성 합성
import ttslearn

In [ ]:
# 시드 고정
from ttslearn.util import init_seed
init_seed(773)

In [ ]:
torch.__version__

### 그래프 그리기 설정 (描画周りの設定) // 번역 수정 필요

In [ ]:
from ttslearn.notebook import get_cmap, init_plot_style, savefig
cmap = get_cmap()
init_plot_style()

### 레시피 설정

In [ ]:
# run.sh를 사용하여 학습 스크립트를 노트북에서 실행하려면 True
# google colab의 경우 True라고 가정합니다.
# 로컬 환경의 경우 run.sh를 터미널에서 실행하는 것이 좋습니다.
# 이 경우이 노트북은 시각화 및 학습 된 모델을 테스트하는 데 사용됩니다.
run_sh = is_colab()

# 참고 : WaveNet을 사용한 평가 데이터에 대한 음성 생성은 시간이 오래 걸립니다.
run_stage8 = True

# run.sh를 통해 실행되는 스크립트의 tqdm
run_sh_tqdm = "none"

# CUDA
# 참고 : run.sh의 인수로 전달하므로 bool이 아닌 문자열로 정의됩니다.
cudnn_benchmark = "true"
cudnn_deterministic = "false"

# 특징량 추출시 병렬 처리 작업 수
n_jobs = os.cpu_count()//2

# 연속 길이 모델의 설정 파일 이름
duration_config_name="duration_rnn"
# 음향 모델 설정 파일 이름
logf0_config_name="logf0_rnn"
# WaveNet 설정 파일 이름
wavenet_config_name="wavenet_sr16k_mulaw256"

# 연속 길이 모델 및 로그 F0 예측 모델 학습의 배치 크기
dnntts_batch_size = 32
# 연속 길이 모델 및 로그 F0 예측 모델 학습의 에포크 수
# 주의: 계산 시간을 줄이기 위해 적게 설정합니다. 품질을 높이려면 30-50 에포크 수를 사용해보십시오.
dnntts_nepochs = 5

# WaveNet 학습의 배치 크기
# 권장 배치 크기 : 8 이상
# 동작 확인을 위해 작은 값으로 설정합니다.
wavenet_batch_size = 4
# WavaNet 학습 반복 수
# 주의: 충분한 품질을 얻기 위해 필요한 값: 300k ~ 500k steps
wavenet_max_train_steps = 50000

# 음성을 생성하는 발화 수
# WaveNet의 추론은 시간이 오래 걸리므로 노트북에서 볼 수 있는 5개만 생성
num_eval_utts = 5

# 노트북에서 사용하는 테스트 발화 (학습 데이터, 평가 데이터)
train_utt = "BASIC5000_0001"
test_utt = "BASIC5000_5000"

### Tensorboard로 로그 시각화

In [ ]:
# 노트북에서 tensorboard 로그를 확인하려면 다음 행을 활성화하십시오.
if is_colab():
    %tensorboard --logdir tensorboard/

## 프로그램 구현 전 준비

### stage -1: 코퍼스 다운로드

In [ ]:
if is_colab():
    ! ./run.sh --stage -1 --stop-stage -1

### Stage 0: 학습/검증/평가 데이터 분할

In [ ]:
if run_sh:
    ! ./run.sh --stage 0 --stop-stage 0

In [ ]:
! ls data/

In [ ]:
! head data/dev.list

## 8.2 데이터 전처리

### 연속 길이 모델에 대한 전처리

배치 처리를 실시하는 커멘드 라인 프로그램은,`recipes/dnntts/preprocess_duration.py`를 참조해 주세요.

In [ ]:
if run_sh:
    ! ./run.sh --stage 1 --stop-stage 1 --n-jobs $n_jobs 

### 로그 F0 예측 모델에 대한 전처리

#### 로그 F0 + 유성/무성 플래그 계산

In [ ]:
from nnmnkwii.preprocessing import delta_features
from nnmnkwii.preprocessing.f0 import interp1d
from ttslearn.dsp import f0_to_lf0

def world_log_f0_vuv(x, sr):
    f0, timeaxis = pyworld.dio(x, sr)
    f0 = pyworld.stonemask(x, f0, timeaxis, sr)
    vuv = (f0 > 0).astype(np.float32)

    # 연속 로그 기본 주파수
    lf0 = f0_to_lf0(f0)
    lf0 = interp1d(lf0)

    # 연속 기본 주파수와 유성/무성 플래그를 2차원 행렬의 형태로 한다.
    lf0 = lf0[:, np.newaxis] if len(lf0.shape) == 1 else lf0
    vuv = vuv[:, np.newaxis] if len(vuv.shape) == 1 else vuv

    # 동적 특징량 계산
    windows = [
        [1.0],  # 정적 특징량에 대한 창
        [-0.5, 0.0, 0.5],  # 1차 동적 특징량에 대한 창
        [1.0, -2.0, 1.0],  # 2차 동적 특징량에 대한 창
    ]
    lf0 = delta_features(lf0, windows)

    # 모든 특징을 결합
    feats = np.hstack([lf0, vuv]).astype(np.float32)

    return feats

#### 로그 F0 + 유성/무성 플래그의 시각화

In [ ]:
from ttslearn.dsp import lf0_to_f0

sr = 16000
_sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768).astype(np.float64)
x = librosa.resample(x, _sr, sr)

out_feats = world_log_f0_vuv(x, sr)
lf0 = out_feats[:, 0]
vuv = out_feats[:, -1]

timeaxis = librosa.frames_to_time(np.arange(len(lf0)), sr, int(0.005 * sr))

fig, ax = plt.subplots(3,1, figsize=(8,6))
ax[0].set_title("Input waveform")
ax[1].set_title("Continuous log F0")
ax[2].set_title("V/UV")

librosa.display.waveshow(x, sr, x_axis="time", ax=ax[0])
ax[1].plot(timeaxis, lf0, linewidth=2)
ax[2].plot(timeaxis, vuv, linewidth=2)

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_xlim(0, len(x)/sr)
    a.set_xticks(np.arange(0, 3.5, 0.5))
    a.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))

ax[0].set_ylabel("Amplitude")
ax[1].set_ylabel("Logarithmic frequency")
ax[2].set_ylabel("Binary value")

plt.tight_layout()

#### 1 발화에 대한 전처리

In [ ]:
from nnmnkwii.frontend import merlin as fe

# HTS 형식의 질문 파일 로드
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 전체 컨텍스트 라벨 로드
labels = hts.load(ttslearn.util.example_label_file())

# 프레임별 언어 특징량 추출
in_feats = fe.linguistic_features(
    labels,
    binary_dict,
    numeric_dict,
    add_frame_features=True,
    subphone_features="coarse_coding",
)

# 오디오 파일 로드
sr = 16000
_sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768).astype(np.float64)
x = librosa.resample(x, _sr, sr)

# 연속 로그 기본 주파수와 유성/무성 플래그를 결합한 특징량 계산
out_feats = world_log_f0_vuv(x.astype(np.float64), sr)

# 프레임 수 조정
minL = min(in_feats.shape[0], out_feats.shape[0])
in_feats, out_feats = in_feats[:minL], out_feats[:minL]

# 시작과 끝의 음성이 아닌 구간의 길이 조정
assert "sil" in labels.contexts[0] and "sil" in labels.contexts[-1]
start_frame = int(labels.start_times[1] / 50000)
end_frame = int(labels.end_times[-2] / 50000)

# 시작과 끝의 음성이 아닌 구간의 길이 조정
start_frame = max(0, start_frame - int(0.050 / 0.005))
end_frame = min(minL, end_frame + int(0.100 / 0.005))

in_feats = in_feats[start_frame:end_frame]
out_feats = out_feats[start_frame:end_frame]

In [ ]:
print ("입력 특징량의 크기:", in_feats.shape)
print ("출력 특징량의 크기:", out_feats.shape)

#### 레시피의 stage 2 실행

위의 처리를 수행하는 일괄 처리 프로그램은 `preprocess_logf0.py`에 있습니다.

In [ ]:
if run_sh:
    ! ./run.sh --stage 2 --stop-stage 2 --n-jobs $n_jobs

### WaveNet을 위한 전처리

#### 1 발화에 대한 전처리

In [ ]:
from ttslearn.dsp import mulaw_quantize
from ttslearn.dsp import world_log_f0_vuv

# HTS 형식의 질문 파일 로드
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 풀 컨텍스트 라벨 로드
labels = hts.load(ttslearn.util.example_label_file())

# 프레임별 언어 특징량 추출
in_feats = fe.linguistic_features(
    labels,
    binary_dict,
    numeric_dict,
    add_frame_features=True,
    subphone_features="coarse_coding",
)

# 오디오 파일 로드
sr = 16000
_sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = (x / 32768).astype(np.float64)
x = librosa.resample(x, _sr, sr)

# 연속 로그 기본 주파수와 유성/무성 플래그를 결합한 특징량 계산
log_f0_vuv = world_log_f0_vuv(x.astype(np.float64), sr)

# 프레임 수 조정
minL = min(in_feats.shape[0], log_f0_vuv.shape[0])
in_feats, log_f0_vuv = in_feats[:minL], log_f0_vuv[:minL]

# 시작과 끝의 음성이 아닌 구간의 길이 조정
assert "sil" in labels.contexts[0] and "sil" in labels.contexts[-1]
start_frame = int(labels.start_times[1] / 50000)
end_frame = int(labels.end_times[-2] / 50000)

# 시작: 50ms, 후미: 100ms
start_frame = max(0, start_frame - int(0.050 / 0.005))
end_frame = min(minL, end_frame + int(0.100 / 0.005))

in_feats = in_feats[start_frame:end_frame]
log_f0_vuv = log_f0_vuv[start_frame:end_frame]

# 언어 특징량과 연속 로그 기본 주파수를 결합
in_feats = np.hstack([in_feats, log_f0_vuv])

# 시간 영역에서 오디오의 길이 조정
x = x[int(start_frame * 0.005 * sr) :]
length = int(sr * 0.005) * in_feats.shape[0]
x = pad_1d(x, length) if len(x) < length else x[:length]

# mu-law 양자화
quantized_x = mulaw_quantize(x)

# 조건부 특징 량의 업 샘플링을 생각하기 위해,
# 음성 파형의 길이가 프레임 시프트로 나누어지는 것을 확인
assert len(quantized_x) % int(sr * 0.005) == 0

In [ ]:
print ("조건부 특징량의 크기:", in_feats.shape)
print ("양자화된 음성 파형의 크기:", quantized_x.shape)

In [ ]:
timeaxis = np.arange(len(x)) / sr

fig, ax = subplots(2,1, figsize=(8,4))
ax[0].set_title("Input waveform")
ax[1].set_title("Output waveform after mu-law")

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Amplitude")
ax[0].plot(timeaxis, x)
ax[1].plot(timeaxis, quantized_x)
plt.tight_layout()

#### 레시피의 stage 3 실행

위의 처리를 수행하는 일괄 처리 프로그램은 `preprocess_wavenet.py`에 있습니다.

In [ ]:
if run_sh:
    ! ./run.sh --stage 3 --stop-stage 3 --n-jobs $n_jobs

### 특징량 정규화

정규화를 위한 통계량을 계산하는 명령행 프로그램은 `recipes/common/fit_scaler.py`를 참조하십시오. 또, 정규화를 실시하는 커멘드 라인 프로그램은, `recipes/common/preprocess_normalize.py` 를 참조해 주세요.

#### 레시피 stage 4 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 4 --stop-stage 4 --n-jobs $n_jobs

#### 정규화 처리 결과 확인

In [ ]:
# 언어 특징량 정규화 전후
in_feats = np.load(f"dump/jsut_sr16000/org/train/in_logf0/{train_utt}-feats.npy")
in_feats_norm = np.load(f"dump/jsut_sr16000/norm/train/in_logf0/{train_utt}-feats.npy")
fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].set_title("Linguistic features (before normalization)")
ax[1].set_title("Linguistic features (after normalization)")
hop_length = int(sr * 0.005)
mesh = librosa.display.specshow(
    in_feats.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames", ax=ax[0], cmap=cmap)
fig.colorbar(mesh, ax=ax[0])
mesh = librosa.display.specshow(
    in_feats_norm.T, sr=sr, hop_length=hop_length, x_axis="time", y_axis="frames",ax=ax[1], cmap=cmap)
# 참고 : 실제로는 [-4, 4] 범위를 벗어난 값이 있지만 가시성을 위해 [-4, 4]로 설정합니다.
mesh.set_clim(-4, 4)
fig.colorbar(mesh, ax=ax[1])

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Context")
    # 후미 비음성 구간 제외
    a.set_xlim(0, 2.55)
    
plt.tight_layout()

## 8.3 연속 길이 모델 학습

### 연속 길이 모델의 설정 파일

In [ ]:
! cat conf/train_dnntts/model/{duration_config_name}.yaml

### 연속 길이 모델 인스턴스화

In [ ]:
import hydra
from omegaconf import OmegaConf
hydra.utils.instantiate(OmegaConf.load(f"conf/train_dnntts/model/{duration_config_name}.yaml").netG)

### 레시피의 stage 5 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 5 --stop-stage 5 --duration-model $duration_config_name \
        --tqdm $run_sh_tqdm --dnntts-data-batch-size $dnntts_batch_size --dnntts-train-nepochs $dnntts_nepochs \
        --cudnn-benchmark $cudnn_benchmark --cudnn-deterministic $cudnn_deterministic

### 손실 함수의 값 추이

저자에 의한 실험 결과입니다. Tensorboard 로그는 https://tensorboard.dev/에 업로드되었습니다.
로그 데이터를 `tensorboard` 패키지를 이용해 다운로드합니다.

https://tensorboard.dev/experiment/yXyg9qgfQRSGxvil5FA4xw/

In [ ]:
if exists("tensorboard/all_log.csv"):
    df = pd.read_csv("tensorboard/all_log.csv")
else:
    experiment_id = "yXyg9qgfQRSGxvil5FA4xw"
    experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
    df = experiment.get_scalars() 
    df.to_csv("tensorboard/all_log.csv", index=False)
df["run"].unique()

In [ ]:
duration_loss = df[df.run.str.contains("duration")]

duration_train_loss = duration_loss[duration_loss.tag.str.contains("Loss/train")]
duration_dev_loss = duration_loss[duration_loss.tag.str.contains("Loss/dev")]

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(duration_train_loss["step"], duration_train_loss["value"], label="Train")
ax.plot(duration_dev_loss["step"], duration_dev_loss["value"], "--", label="Dev")
ax.set_xlabel("Epoch")
ax.set_ylabel("Epoch loss")
plt.legend()

# 그림 8-3
savefig("fig/wavenet_impl_duration_loss")

## 8.4 로그 F0 예측 모델 학습

### 로그 F0 예측 모델 설정 파일

In [ ]:
! cat conf/train_dnntts/model/{logf0_config_name}.yaml

### 로그 F0 예측 모델 인스턴스화

In [ ]:
import hydra
from omegaconf import OmegaConf
hydra.utils.instantiate(OmegaConf.load(f"conf/train_dnntts/model/{logf0_config_name}.yaml").netG)

### 레시피 stage 6 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 6 --stop-stage 6 --logf0-model $logf0_config_name \
        --tqdm $run_sh_tqdm --dnntts-data-batch-size $dnntts_batch_size --dnntts-train-nepochs $dnntts_nepochs \
        --cudnn-benchmark $cudnn_benchmark --cudnn-deterministic $cudnn_deterministic

### 손실 함수의 값 추이

In [ ]:
logf0_loss = df[df.run.str.contains("logf0")]

logf0_train_loss = logf0_loss[logf0_loss.tag.str.contains("Loss/train")]
logf0_dev_loss = logf0_loss[logf0_loss.tag.str.contains("Loss/dev")]

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(logf0_train_loss["step"], logf0_train_loss["value"], label="Train")
ax.plot(logf0_dev_loss["step"], logf0_dev_loss["value"], "--", label="Dev")
ax.set_xlabel("Epoch")
ax.set_ylabel("Epoch loss")
plt.legend()

# 그림 8-4
savefig("fig/wavenet_impl_logf0_loss")

## 8.5 WaveNet 학습 스크립트 구현

### DataLoader 구현

#### collate_fn 구현

In [ ]:
def collate_fn_wavenet(batch, max_time_frames=100, hop_size=80, aux_context_window=2):
    max_time_steps = max_time_frames * hop_size

    xs, cs = [b[1] for b in batch], [b[0] for b in batch]

    # 조건부 특징 량의 시작 위치를 무작위로 추출한 후 해당하는 짧은 음성 파형을 잘라냅니다.
    c_lengths = [len(c) for c in cs]
    start_frames = np.array(
        [
            np.random.randint(
                aux_context_window, cl - aux_context_window - max_time_frames
            )
            for cl in c_lengths
        ]
    )
    x_starts = start_frames * hop_size
    x_ends = x_starts + max_time_steps
    c_starts = start_frames - aux_context_window
    c_ends = start_frames + max_time_frames + aux_context_window
    x_batch = [x[s:e] for x, s, e in zip(xs, x_starts, x_ends)]
    c_batch = [c[s:e] for c, s, e in zip(cs, c_starts, c_ends)]

    # numpy.ndarray 의 리스트형으로부터 torch.Tensor 형에 변환합니다
    x_batch = torch.tensor(x_batch, dtype=torch.long)  # (B, T)
    c_batch = torch.tensor(c_batch, dtype=torch.float).transpose(2, 1)  # (B, C, T')

    return x_batch, c_batch

#### DataLoader 사용 예

In [ ]:
from pathlib import Path
from ttslearn.train_util import Dataset
from functools import partial

in_paths = sorted(Path("./dump/jsut_sr16000/norm/dev/in_wavenet/").glob("*.npy"))
out_paths = sorted(Path("./dump/jsut_sr16000/org/dev/out_wavenet/").glob("*.npy"))

dataset = Dataset(in_paths, out_paths)
collate_fn = partial(collate_fn_wavenet, max_time_frames=100, hop_size=80, aux_context_window=0)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=8, collate_fn=collate_fn, num_workers=0)

wavs, feats = next(iter(data_loader))

print("음성 파형의 크기:", tuple(wavs.shape))
print("조건부 특징량의 크기:", tuple(feats.shape))

#### 미니 배치 시각화

In [ ]:
from ttslearn.dsp import inv_mulaw_quantize

fig, ax = plt.subplots(len(wavs), 1, figsize=(8,10), sharex=True, sharey=True)
for n in range(len(wavs)):
    x = wavs[n].data.numpy()
    x = inv_mulaw_quantize(x, 255)
    ax[n].plot(x)

ax[-1].set_xlabel("Time [sample]")
for a in ax:
    a.set_ylabel("Amplitude")
plt.tight_layout()

# 그림 8-5
savefig("fig/wavenet_impl_minibatch")

### 간단한 학습 스크립트 구현

#### 모델 파라미터의 지수 이동 평균

In [ ]:
def moving_average_(model, model_test, beta=0.9999):
    for param, param_test in zip(model.parameters(), model_test.parameters()):
        param_test.data = torch.lerp(param.data, param_test.data, beta)

#### 학습 전 준비

In [ ]:
from ttslearn.wavenet import WaveNet
from torch import optim

# 동작 확인용: 레이어 수를 줄인 작은 WaveNet
ToyWaveNet = partial(WaveNet, out_channels=256, layers=2, stacks=1, kernel_size=2, cin_channels=333)

model = ToyWaveNet()
# 모델 파라미터의 지수 이동 평균
model_ema = ToyWaveNet()
model_ema.load_state_dict(model.state_dict())

# lr은 학습률을 나타냅니다.
optimizer = optim.Adam(model.parameters(), lr=0.01)

# gamma는 학습률의 감쇠 계수를 나타냅니다.
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, gamma=0.5, step_size=100000)

#### 学習ループの実装

In [ ]:
# DataLoader를 사용하여 미니 배치 생성 : 미니 배치마다 처리
for x, c in data_loader:
    # 순전파 계산
    x_hat = model(x, c)
    # 음의 로그 우도(likelihood) 계산
    loss = nn.CrossEntropyLoss()(x_hat[:, :, :-1], x[:, 1:]).mean()
    # 손실 값을 출력
    print(loss.item())
    # optimizer에 축적 된 기울기를 재설정
    optimizer.zero_grad()
    # 오차의 역전파 계산
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 이동 지수 평균 계산
    moving_average_(model, model_ema)
    # 학습률 스케줄러 업데이트
    lr_scheduler.step()

### 실용적인 학습 스크립트 구현

`train_wavenet.py`를 참조하십시오.

## 8.6 WaveNet 학습

### WaveNet 구성 파일

In [ ]:
! cat conf/train_wavenet/model/{wavenet_config_name}.yaml

### WaveNet 인스턴스화

In [ ]:
import hydra
from omegaconf import OmegaConf
# WaveNet의 30층 모두를 표시하면 길어지므로 여기서는 생략합니다.
# hydra.utils.instantiate(OmegaConf.load(f"./conf/train_wavenet/model/{wavenet_config_name}.yaml")["netG"])

### 레시피의 stage 7 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 7 --stop-stage 7 --wavenet-model $wavenet_config_name \
        --tqdm $run_sh_tqdm --wavenet-data-batch-size $wavenet_batch_size --wavenet-train-max-train-steps $wavenet_max_train_steps \
        --cudnn-benchmark $cudnn_benchmark --cudnn-deterministic $cudnn_deterministic

### 손실 함수의 값 추이

In [ ]:
wavenet_loss = df[df.run.str.contains("wavenet")]

wavenet_train_loss = wavenet_loss[wavenet_loss.tag.str.contains("Loss/train")]
wavenet_dev_loss = wavenet_loss[wavenet_loss.tag.str.contains("Loss/dev")]

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(wavenet_train_loss["step"], wavenet_train_loss["value"], label="Train")
ax.plot(wavenet_dev_loss["step"], wavenet_dev_loss["value"], "--", label="Dev")
ax.set_xlabel("Epoch")
ax.set_ylabel("Epoch loss")
ax.set_ylim(1.7, 2.2)
plt.legend()

# 그림 8-6
savefig("fig/wavenet_impl_wavenet_loss")

## 8.7 학습된 모델을 사용하여 텍스트에서 음성 합성

### 학습된 모델 로드

In [ ]:
import joblib
device = torch.device("cpu")

#### 연속 길이 모델 로드

In [ ]:
duration_config = OmegaConf.load(f"exp/jsut_sr16000/{duration_config_name}/model.yaml")
duration_model = hydra.utils.instantiate(duration_config.netG)
checkpoint = torch.load(f"exp/jsut_sr16000/{duration_config_name}/latest.pth", map_location=device)
duration_model.load_state_dict(checkpoint["state_dict"])
duration_model.eval();

#### 로그 F0 예측 모델 로드

In [ ]:
logf0_config = OmegaConf.load(f"exp/jsut_sr16000/{logf0_config_name}/model.yaml")
logf0_model = hydra.utils.instantiate(logf0_config.netG)
checkpoint = torch.load(f"exp/jsut_sr16000/{logf0_config_name}/latest.pth", map_location=device)
logf0_model.load_state_dict(checkpoint["state_dict"])
logf0_model.eval();

#### WaveNet 로드

In [ ]:
wavenet_config = OmegaConf.load(f"exp/jsut_sr16000/{wavenet_config_name}/model.yaml")
wavenet_model = hydra.utils.instantiate(wavenet_config.netG)
checkpoint = torch.load(f"exp/jsut_sr16000/{wavenet_config_name}/latest_ema.pth", map_location=device)
wavenet_model.load_state_dict(checkpoint["state_dict"])
# weight normalization 은 추론시에는 불필요하므로 제외
wavenet_model.remove_weight_norm_()
wavenet_model.eval();

#### 통계량 로드

In [ ]:
duration_in_scaler = joblib.load("./dump/jsut_sr16000/norm/in_duration_scaler.joblib")
duration_out_scaler = joblib.load("./dump/jsut_sr16000/norm/out_duration_scaler.joblib")
logf0_in_scaler = joblib.load("./dump/jsut_sr16000/norm/in_logf0_scaler.joblib")
logf0_out_scaler = joblib.load("./dump/jsut_sr16000/norm/out_logf0_scaler.joblib")
wavenet_in_scaler = joblib.load("./dump/jsut_sr16000/norm/in_wavenet_scaler.joblib")

### 음소 연속 길이 예측

In [ ]:
from ttslearn.util import lab2phonemes, find_lab, find_feats
from ttslearn.dnntts.gen import predict_duration

labels = hts.load(find_lab("downloads/jsut_ver1.1/", test_utt))

# 풀 컨텍스트 라벨에서 음소만 추출
test_phonemes = lab2phonemes(labels)

# 언어 특징량 추출에 사용하기 위한 질문 파일
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

# 음소 연속 길이 예측
durations_test = predict_duration(
    device, labels, duration_model, duration_config, duration_in_scaler, duration_out_scaler,
    binary_dict, numeric_dict)
durations_test_target = np.load(find_feats("dump/jsut_sr16000/org", test_utt, typ="out_duration"))

fig, ax = plt.subplots(1,1, figsize=(6,4))
ax.plot(durations_test_target, "-+", label="Target")
ax.plot(durations_test, "--*", label="Predicted")
ax.set_xticks(np.arange(len(test_phonemes)))
ax.set_xticklabels(test_phonemes)
ax.set_xlabel("Phoneme")
ax.set_ylabel("Duration (the number of frames)")
ax.legend()

plt.tight_layout()

### 로그 기본 주파수 예측

In [ ]:
from ttslearn.dnntts.gen import predict_acoustic

labels = hts.load(find_lab("downloads/jsut_ver1.1/", test_utt))
# 로그 기본 주파수 예측
out_feats = predict_acoustic(
    device, labels, logf0_model, logf0_config, logf0_in_scaler,
    logf0_out_scaler, binary_dict, numeric_dict)

In [ ]:
from ttslearn.util import trim_silence
from ttslearn.dnntts.multistream import split_streams

# 결합된 특징량 분리
out_feats = trim_silence(out_feats, labels)
lf0_gen, vuv_gen = out_feats[:, 0], out_feats[:, 1]

In [ ]:
from ttslearn.dnntts.multistream import get_static_features

# 비교를 위해 자연음성에서 추출한 음향 특징량을 읽기
feats = np.load(find_feats("dump/jsut_sr16000/org/", test_utt, typ="out_logf0"))
# 특징량 분리
lf0_ref, vuv_ref = get_static_features(
    feats, logf0_config.num_windows, logf0_config.stream_sizes, logf0_config.has_dynamic_features)

#### F0 시각화

In [ ]:
# 로그 기본 주파수에서 기본 주파수로 변환
f0_ref = np.exp(lf0_ref)
f0_ref[vuv_ref < 0.5] = 0
f0_gen = np.exp(lf0_gen)
f0_gen[vuv_gen < 0.5] = 0

timeaxis = librosa.frames_to_time(np.arange(len(f0_ref)), sr=sr, hop_length=int(0.005 * sr))

fix, ax = plt.subplots(1,1, figsize=(8,3))
ax.plot(timeaxis, f0_ref, linewidth=2, label="F0 of natural speech")
ax.plot(timeaxis, f0_gen, "--", linewidth=2, label="F0 of generated speech")

ax.set_xlabel("Time [sec]")
ax.set_ylabel("Frequency [Hz]")
ax.set_xlim(timeaxis[0], timeaxis[-1])

plt.legend()
plt.tight_layout()

### 음성 파형 생성

In [ ]:
from ttslearn.dsp import inv_mulaw_quantize

@torch.no_grad()
def gen_waveform(
    device,  # cpu or cuda
    labels,  # 풀 컨텍스트 라벨
    logf0_vuv,  # 연속 로그 기본 주파수 및 유성/무성 플래그
    wavenet_model,  # 학습된 WaveNet
    wavenet_in_scaler,  # 조건부 특징의 정규화를위한 StandardScaler
    binary_dict,  # 이진 특징량을 추출하는 정규식
    numeric_dict,  # 수치 특징량을 추출하는 정규 표현
    tqdm=tqdm,  # 프로그래스바
):
    # 프레임별 언어 특징량 추출
    in_feats = fe.linguistic_features(
        labels,
        binary_dict,
        numeric_dict,
        add_frame_features=True,
        subphone_features="coarse_coding",
    )
    # 프레임 단위의 언어 특징량과 로그 연속 기본 주파수·유성/무성 플래그를 결합
    in_feats = np.hstack([in_feats, logf0_vuv])

    # 특징량 정규화
    in_feats = wavenet_in_scaler.transform(in_feats)

    # 조건부 특징을 numpy.ndarray에서 torch.Tensor로 변환
    c = torch.from_numpy(in_feats).float().to(device)
    # (B, T, C) -> (B, C, T)
    c = c.view(1, -1, c.size(-1)).transpose(1, 2)

    # 음성 파형의 길이 계산
    upsample_scale = np.prod(wavenet_model.upsample_scales)
    time_steps = (c.shape[-1] - wavenet_model.aux_context_window * 2) * upsample_scale

    # WaveNet으로 음성 파형 생성
    # 참고 : 계산에 시간이 걸리기 위해 tqdm의 진행률 막대를 사용합니다.
    gen_wav = wavenet_model.inference(c, time_steps, tqdm)

    # One-hot 벡터를 1차원 신호로 변환
    gen_wav = gen_wav.max(1)[1].float().cpu().numpy().reshape(-1)

    # Mu-law 양자화의 역변환
    gen_wav = inv_mulaw_quantize(gen_wav, wavenet_model.out_channels - 1)

    return gen_wav

### 모든 모델을 결합하여 음성 파형 생성

In [ ]:
# NOTE: False의 경우 정답의 durations를 사용합니다.
# 모든 모델을 연결하려면 True로 설정하십시오.
use_ground_truth_durations = True

labels = hts.load(find_lab("downloads/jsut_ver1.1/", test_utt))

# 언어 특징량 추출의 사전 준비
binary_dict, numeric_dict = hts.load_question_set(ttslearn.util.example_qst_file())

if not use_ground_truth_durations:
    # 음소 연속 길이 예측
    durations = predict_duration(
        device, labels, duration_model, duration_config, duration_in_scaler, duration_out_scaler,
        binary_dict, numeric_dict)

    # 예측된 연속 길이를 풀 컨텍스트 레이블로 설정
    labels.set_durations(durations)

# 로그 기본 주파수 예측
# 참고 : 동적 특징 량을 WaveNet 조건부 특징 량에 사용하기 때문에 매개 변수 생성 (mlpg)을 수행하지 않습니다.
logf0_vuv = predict_acoustic(
    device, labels, logf0_model, logf0_config, logf0_in_scaler,
    logf0_out_scaler, binary_dict, numeric_dict, mlpg=False)

# WaveNet으로 음성 파형 생성
gen_wav = gen_waveform(
    device, labels, logf0_vuv, wavenet_model, wavenet_in_scaler,
    binary_dict, numeric_dict, tqdm)

In [ ]:
# 비교를 위해 원래 음성 로드
from scipy.io import wavfile
_sr, ref_wav = wavfile.read(f"./downloads/jsut_ver1.1/basic5000/wav/{test_utt}.wav")
ref_wav = (ref_wav / 32768.0).astype(np.float64)
ref_wav = librosa.resample(ref_wav, _sr, sr)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8,6))

hop_length = int(sr * 0.005)
fft_size = pyworld.get_cheaptrick_fft_size(sr)

spec_ref = librosa.stft(ref_wav, n_fft=fft_size, hop_length=hop_length, window="hann")
logspec_ref = np.log(np.abs(spec_ref))
spec_gen = librosa.stft(gen_wav, n_fft=fft_size, hop_length=hop_length, window="hann")
logspec_gen = np.log(np.abs(spec_gen))

mindb = min(logspec_ref.min(), logspec_gen.min())
maxdb = max(logspec_ref.max(), logspec_gen.max())

mesh = librosa.display.specshow(logspec_ref, hop_length=hop_length, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax[0])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[0], format="%+2.fdB")

mesh = librosa.display.specshow(logspec_gen, hop_length=hop_length, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax[1])
mesh.set_clim(mindb, maxdb)
fig.colorbar(mesh, ax=ax[1], format="%+2.fdB")

ax[0].set_title("Spectrogram of natural speech")
ax[1].set_title("Spectrogram of generated speech")
for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")

plt.tight_layout()

print("自然音声")
IPython.display.display(Audio(ref_wav, rate=sr))
print("WaveNet音声合成")
IPython.display.display(Audio(gen_wav, rate=sr))

# 그림 8-7
savefig("./fig/wavenet_impl_tts_spec_comp")

### 평가 데이터에 대한 음성 파형 생성

#### 레시피의 stage 8 실행

In [ ]:
if run_sh and run_stage8:
    ! ./run.sh --stage 8 --stop-stage 8 \
        --tqdm $run_sh_tqdm --duration-model $duration_config_name --logf0-model $logf0_config_name --wavenet-model $wavenet_config_name \
        --reverse true --num-eval-utts $num_eval_utts

## 자연 음성과 합성 음성의 비교 (bonus)

In [ ]:
from pathlib import Path
from ttslearn.util import load_utt_list

with open("./downloads/jsut_ver1.1/basic5000/transcript_utf8.txt") as f:
    transcripts = {}
    for l in f:
        utt_id, script = l.split(":")
        transcripts[utt_id] = script
        
eval_list = load_utt_list("data/eval.list")[::-1][:5]

for utt_id in eval_list:
    # ref file 
    ref_file = f"./downloads/jsut_ver1.1/basic5000/wav/{utt_id}.wav"
    _sr, ref_wav = wavfile.read(ref_file)
    ref_wav = (ref_wav / 32768.0).astype(np.float64)
    ref_wav = librosa.resample(ref_wav, _sr, sr)
    
    print(f"{utt_id}: {transcripts[utt_id]}")
    print("自然音声")
    IPython.display.display(Audio(ref_wav, rate=sr))

    gen_file = f"exp/jsut_sr16000/synthesis_{duration_config_name}_{logf0_config_name}_{wavenet_config_name}/eval/{utt_id}.wav"
    if exists(gen_file):
        _sr, gen_wav = wavfile.read(gen_file)    
        print("WaveNet音声合成")
        IPython.display.display(Audio(gen_wav, rate=sr))
    else:
        # 음성 생성이 완료되지 않은 경우
        print("WaveNet音声合成: not found")

## 학습된 모델 패키징 (bonus)

학습된 모델을 사용하는 TTS에 필요한 모든 파일을 단일 디렉토리로 그룹화합니다.
`ttslearn.wavenet.WaveNetTTS` 클래스에는, 정리한 디렉토리를 지정해, TTS를 실시하는 기능이 구현되고 있습니다.

### 레시피의 stage 99 실행

In [ ]:
if run_sh:
    ! ./run.sh --stage 99 --stop-stage 99 \
        --duration-model $duration_config_name --logf0-model $logf0_config_name --wavenet-model $wavenet_config_name

In [ ]:
!ls tts_models/jsut_sr16000_{duration_config_name}_{logf0_config_name}_{wavenet_config_name}

### 패키징된 모델을 이용한 TTS

In [ ]:
from ttslearn.wavenet import WaveNetTTS

# 패키징된 모델의 경로를 지정합니다.
engine = WaveNetTTS(
    model_dir=f"./tts_models/jsut_sr16000_{duration_config_name}_{logf0_config_name}_{wavenet_config_name}"
)
wav, sr = engine.tts("ここまでお読みいただき、ありがとうございました。", tqdm=tqdm)

fig, ax = plt.subplots(figsize=(8,2))
librosa.display.waveshow(wav.astype(np.float32), sr, ax=ax)
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

Audio(wav, rate=sr)

In [ ]:
if is_colab():
    from datetime import timedelta
    elapsed = (time.time() - start_time)
    print("소요시간:", str(timedelta(seconds=elapsed)))